# Array Manipulation and Linear Algebra in BART

BART offers a large suite of numerical array manipulation and linear algebra tools. In this notebook, we will explore some of this array processing.

* Full Tutorials
    * [x] extract
    * [x] flatten
    * [x] flip
    * [x] invert
    * [x] noise
    * [x] normalize
    * [x] repmat
    * [ ] saxpy
    * [x] scale
    * [ ] sdot
    * [x] std
    * [ ] svd
    * [x] transpose
    * [x] var
    * [x] vec
    
* **TODO**
    * flags for normalize

In [186]:
cd bart
. startup.sh
cd ..

Usage: saxpy scale <input1> <input2> <output>

Multiply input1 with scale factor and add input2.

-h		help


In [2]:
bart

BART. Available commands are:
avg         bench       bitmask     cabs        caldir      calmat      
carg        casorati    cc          ccapply     cdf97       circshift   
conj        conv        copy        cpyphs      creal       crop        
delta       ecalib      ecaltwo     estdelay    estdims     estshift    
estvar      extract     fakeksp     fft         fftmod      fftshift    
filter      flatten     flip        fmac        homodyne    invert      
itsense     join        lrmatrix    mandelbrot  mip         nlinv       
noise       normalize   nrmse       nufft       ones        pattern     
phantom     pics        pocsense    poisson     repmat      reshape     
resize      rof         rss         sake        saxpy       scale       
sdot        show        slice       spow        sqpics      squeeze     
std         svd         threshold   toimg       traj        transpose   
twixread    var         vec         version     walsh       wave        
wavelet     wavepsf  

: 1

## Generating Values

### `bart ones`, `bart zeros` and `bart vec`



In [52]:
bart ones -h

Usage: ones dims dim1 ... dimn name

Create an array filled with ones with {dims} dimensions of size {dim1} to {dimn}.

-h		help


`ones` generates an array of all 1's. Similarly, `zeros` gives us an all-zero array of the specified dimensions 

In [66]:
bart ones 1 5 five_ones
bart zeros 1 5 five_zeros

Note that the `show` utility allows us to view the generated values

In [68]:
bart show -h

Usage: show [-m] [-d d] [-s <string>] [-f <string>] <input>

Outputs values or meta data.

-m		show meta data
-d dim      	show size of dimension
-s sep      	use <sep> as the separator
-f format      	use <format> as the format. Default: "%+e%+ei"
-h		help


In [81]:
## TODO Truncate Format String

bart show five_ones

bart show five_zeros

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i	+0.000000e+00+0.000000e+00i


To create a vector of specific values, we can use `bart vec`:

In [130]:
bart vec 1 2 3 4 5 test_vec

bart show test_vec

+1.000000e+00+0.000000e+00i	+2.000000e+00+0.000000e+00i	+3.000000e+00+0.000000e+00i	+4.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i


## Basic Array Manipulation

### `bart normalize`, `bart scale`, `bart invert`, `bart flip`

Let's normalize our test vector from above

In [104]:
bart normalize -h

Usage: normalize flags <input> <output>

Normalize along selected dimensions.

-h		help


Note that we use `-b` to specify a bitmask

In [112]:
bart normalize -b 0 test_vec normalized_vec

bart show normalized_vec

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


Now, we are left with a normalized (unit) vector. Using `bart scale` we can apply a real or complex scalar.

In [107]:
bart scale -h

Usage: scale factor <input> <output>

Scale array by {factor}. The scale factor can be a complex number.

-h		help


In [108]:
bart scale 5 normalized_vec scaled_vec

bart show scaled_vec

+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i	+5.000000e+00+0.000000e+00i


In [111]:
bart scale 1+1i normalized_vec scaled_complex_vec

bart show scaled_complex_vec

+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i	+1.000000e+00+1.000000e+00i


Important: When using a complex number, the imaginary coefficient *must* be included. BART will not recognize `1 + i` as a valid scalar, but `1 + 1i`, as shown above, works perfectly. 

Next, we will use `invert` to perform elementwise inversion of our vector.

In [126]:
bart invert -h 

Usage: invert <input> <output>

Invert array (1 / <input>). The output is set to zero in case of divide by zero.

-h		help


In [129]:
bart invert test_vec inverted_vec

bart show inverted_vec

+1.000000e+00+0.000000e+00i	+5.000000e-01+0.000000e+00i	+3.333333e-01+0.000000e+00i	+2.500000e-01+0.000000e+00i	+2.000000e-01+0.000000e+00i


As described, each element `x` will be replaced by `1 / x`. In the image processing tutorial, we explore the effect of this elementwise transformation on visual data.

With `flip` we can flip the indices of an array along some dimension. 

In [206]:
bart flip -h

Usage: flip bitmask <input> <output>

Flip (reverse) dimensions specified by the {bitmask}.

-h		help


In [208]:
bart flip 1 test_vec flipped_vec

bart show flipped_vec

+5.000000e+00+0.000000e+00i	+4.000000e+00+0.000000e+00i	+3.000000e+00+0.000000e+00i	+2.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


## Summary Statistics

### `bart std`, `bart var`

First, we use `bart noise` to apply real-valued noise with a variance of 4 to a vector of all zeros.

In [165]:
bart noise -h

Usage: noise [-s d] [-r] [-n f] <input> <output>

Add noise with selected variance to input.

-s random seed initialization
-r		real-valued input
-n variance      	DEFAULT: 1.0
-h		help


In [173]:
bart ones 1 10 zeros
bart noise -n 4 -r zeros noisy_vec

bart show noisy_vec

+5.672054e-01+0.000000e+00i	+9.672512e-02+0.000000e+00i	+4.566258e+00+0.000000e+00i	-3.144541e+00+0.000000e+00i	-2.936252e+00+0.000000e+00i	+1.935789e+00+0.000000e+00i	-4.396978e+00+0.000000e+00i	+1.351878e+00+0.000000e+00i	+5.980763e-01+0.000000e+00i	+2.039012e+00+0.000000e+00i


Now, we will explore some summary statistics of our noisy data. Note that we use a bitmask to specify the dimensions across which to average or take the variance of. 

In [174]:
bart std $(bart bitmask 0) noisy_vec std

bart show std

+2.770262e+00+0.000000e+00i


In [175]:
bart var $(bart bitmask 0) noisy_vec var

bart show var

+7.674354e+00+0.000000e+00i


## Multi-Dimensional Data Manipulation

### `bart extract`, `bart flatten`, `bart repmat`, `bart transpose`

In [188]:
bart ones 2 3 4 ones_matrix

We'll create a demo $3 x 4$ matrix of ones to demonstrate the use of these functions.

In [189]:
bart show ones_matrix

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


First, we'll use `extract` to isolate the first two columns. We can specify a dimension (`0`) and appropriately index the columns (`0` to `2`)

In [194]:
bart extract -h

Usage: extract dimension start end <input> <output>

Extracts a sub-array along {dim} from index {start} to (not including) {end}.

-h		help


In [197]:
bart extract 0 0 2 ones_matrix ones_extracted

bart show ones_extracted

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


Next, we'll use `repmat` to replicate our matrix along a specified dimension

In [221]:
bart repmat -h

Usage: repmat dimension repetitions <input> <output>

Repeat input array multiple times along a certain dimension.

-h		help


In [234]:
bart repmat 2 3 ones_matrix ones_repeated

Now observe what happens if we take `extract` a slice of length 1 along dimension 2

In [237]:
bart extract 2 0 1 ones_repeated ones_slice

bart show ones_slice

+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i
+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i	+1.000000e+00+0.000000e+00i


We have successfully replicated the array along dimension 2! 

Lastly, we can use `transpose` to perform a simple transpose operation

In [240]:
bart transpose 0 1 ones_matrix transposed_ones

bart show -m ones_matrix
bart show -m transposed_ones

Type: complex float
Dimensions: 16
AoD:	3	4	1	1	1	1	1	1	1	1	1	1	1	1	1	1
Type: complex float
Dimensions: 16
AoD:	4	3	1	1	1	1	1	1	1	1	1	1	1	1	1	1


The first set of metadata is that which corresponds to the original matrix. The second corresponds to its transpose.

## Linear Algebra Functionality

### `bart saxpy`, `bart sdot`, `bart svd`

`saxpy` ('scalar a x plus y') computes the following scaled vector sum: $$\vec{z} = {a}\vec{x} + \vec{y}$$

In [241]:
bart saxpy -h

Usage: saxpy scale <input1> <input2> <output>

Multiply input1 with scale factor and add input2.

-h		help


In [251]:
bart vec 1 2 3 4 5 test_vec
bart saxpy 5 test_vec test_vec saxpy_out

bart show saxpy_out

+6.000000e+00+0.000000e+00i	+1.200000e+01+0.000000e+00i	+1.800000e+01+0.000000e+00i	+2.400000e+01+0.000000e+00i	+3.000000e+01+0.000000e+00i


`sdot` provides standard dot-product functionality

In [252]:
bart sdot -h

Usage: sdot <input1> <input2>

Compute dot product along selected dimensions.

-h		help


In [253]:
bart sdot test_vec test_vec

+5.500000e+01+0.000000e+00i


Lastly, `bart svd` allows us to compute the SIngular Value Decomposition of a given matrix

In [255]:
bart svd -h

Usage: svd [-e] <input> <U> <S> <VH>

Compute singular-value-decomposition (SVD).


-e		econ
-h		help


In [256]:
bart ones 2 4 3 ones_matrix

bart svd ones_matrix ones_u ones_s ones_vh

In [263]:
echo "U:"
bart show ones_u

echo "Sigma:"
bart show ones_s 

echo "V*:"
bart show ones_vh

U:
-5.000001e-01+0.000000e+00i	-5.000000e-01+0.000000e+00i	-5.000000e-01+0.000000e+00i	-5.000000e-01+0.000000e+00i
+8.660254e-01+0.000000e+00i	-2.886751e-01+0.000000e+00i	-2.886751e-01+0.000000e+00i	-2.886751e-01+0.000000e+00i
+0.000000e+00+0.000000e+00i	-5.773503e-01+0.000000e+00i	+7.886751e-01+0.000000e+00i	-2.113249e-01+0.000000e+00i
+0.000000e+00+0.000000e+00i	-5.773503e-01+0.000000e+00i	-2.113249e-01+0.000000e+00i	+7.886751e-01+0.000000e+00i
Sigma:
+3.464102e+00+0.000000e+00i	+1.685874e-07+0.000000e+00i	+0.000000e+00+0.000000e+00i
V*:
-5.773503e-01+0.000000e+00i	-8.164966e-01+0.000000e+00i	+0.000000e+00+0.000000e+00i
-5.773503e-01+0.000000e+00i	+4.082482e-01+0.000000e+00i	-7.071068e-01+0.000000e+00i
-5.773503e-01+0.000000e+00i	+4.082483e-01+0.000000e+00i	+7.071067e-01+0.000000e+00i
